## Functions and basic types

We will continue the RPG game from lesson 3 and dive deeper into functions and basic types.  

A function is declared with `def` followed by a name, and then a parenthesis with 0 or more parameters.

Python functions have several special abilities:

- Can take default values for parameters
- Can specify keyword at the calling of a function
- Can take positional only parameters using `/`
- Can take keyword only parameters using `*`
- Can pass in a list for positional args with the `*identifier` (typically `*args`) operator
    - If specified, must come before the `*` marker
- Can pass in a dictionary for keyword args using the `**identifier` (typically `**kwargs`) operator
    - if `*args` is used, must come after it

All these rules can make it pretty complicated pretty fast.  I recommend sticking to a subset of these.  It is often
useful to specify by kw-only even if more verbose.  But to show you how complicated it can get here are some examples
that we will progressively expand on

## Simple function

Above we have a basic function `die` with type annotations.  It takes a single arg called `size` which is of type `int`.
It returns a `int` and is denoted by `-> int:`.  

> Note that in every modern language since about 2010, all langauges now have the type to the right of the name of the
variable.  This is opposite of C family languages like java.  This is because it is easier for the parser to do type
inference when the type is the right of the variable.  This is true for the return type as well (if you think of the
function declaration as the "name", the return type is also to the right of the declaration).  So get used to this as
all languages since 2010 have been doing it this way.

In python, types are purely optional:

- They are are validated by a static type checker (in your IDE or as part of a CI check) 
- At runtime, the python interpreter does not use any of this information
    - The annotations are available with reflection though
    - a few python libraries actually require type annotations like FastApi and Pydantic.

However, you should _always_ use types, even if they optional.

In [ ]:
import random as rand

def die(size: int) -> int:
    """Generates a random number from 1 - size (inclusize)

    Parameters
    ----------
    size : int
        The size of the die (eg, 4, 6, 8, 10, 12, 20, etc)

    Returns
    -------
    int
        the random result of the roll
    """
    return rand.randint(1, size)

# Can call it without naming the variable
print(die(20))
# or by naming the variable
die(size=10)


## Functions with default args

The second function is called `die_w_default` and notice that after the `{name}: {type}` there is `= 20`.  This is how
one defines a default value for a function or method.  Using default args comes with some caveats:

- A defaulted arg must come after all non-defaulted args (see `dice` function)
- Do not use a default arg which is a mutable reference (eg, an empty list)

The latter caveat bears some explanation.  In python, the default argument is created **once** when the function is 
defined.  It is not generated each time a function is invoked.  Let's see what happens if we try

In [ ]:
def die_w_default(size: int = 20) -> int:
    """Returns a random number from [1, size] inclusive

    Parameters
    ----------
    size : int, optional
        the die size, by default 20

    Returns
    -------
    int
        the result of the roll
    """
    return die(size)

#def die_w_default_bad(size: int = 10, number):
#    ...

def dice(number: int, size: int = 20) -> list[int]:
    """Rolls `number` of dice of type `size`

    Parameters
    ----------
    number : int
        the number of dice to roll
    size : int, optional
        the size of the die type, by default 20

    Returns
    -------
    list[int]
        the result of the die rolls
    """
    
    return [die(size) for _ in range(number)]

# Can call it without naming the variable
print(die_w_default(10))
# Or with naming it
print(die_w_default(size=12))

result = die_w_default()
print(f"result with default = {result}")

# same with dice
print(dice(3, 6))
print(dice(number=4, size=20))
print(dice(3, size=10))
print(dice(4))
# However, if you name a variable, it must come positional only args
#print(dice(number=4, 20)) # won't work
print(dice(4, size=20))

## Do not use defaults that are mutable

One gotcha with defaulted parameters is that the defaulted value is created once when python first encounters the 
function definition, and that initial defaulted value gets reused on all new calls.  We can see an example of this
behavior like this.

In [ ]:
# This is a bad example, with a mutable default with an empty list
def mutable_default(num_rolls: int, rolls: list[int] = []) -> list[int]:
    for _ in range(num_rolls):
        result = dice(3, 6)
        rolls.extend(result)
    return rolls

# Let's make 1 roll
rolls = mutable_default(1)
print(rolls)
# and run again using the default
second_roll = mutable_default(1)
print(second_roll) # hmm, why does it contain the second?

third_roll = mutable_default(1, [])
print(third_roll)

print(second_roll)

In [ ]:
# Fixed version
def fixed_mutable_default(num_rolls: int, rolls: list[int] | None = None) -> list[int]:
    if rolls is None:
        rolls = []
    for _ in range(num_rolls):
        result = dice(3, 6)
        rolls.extend(result)
    return rolls

# Let's make 1 roll
rolls = fixed_mutable_default(1)
print(rolls)
# and run again using the default
second_roll = fixed_mutable_default(1)
print(second_roll) 

## Postional and keyword only arguments

Sometimes, when there are many arguments to a function or method, it's good to specify which arguments are positional
and which arguments must be specified by keyword.  As shown from the examples above, you could call any of the functions
either positionally, or with the name of the keyword.  To change this behavior you can do this:

- Use `/` in your argument list to specify that any _args to the left_ are positional only
- Use `*` in your argument list to specify that any _args to the right_ are keyword only

Let's modify dice so that you must specify the `number` as a positional arg, and size as a keyword arg

In [ ]:
def dice(number: int, /, *, size: int = 20) -> list[int]:
    return [die(size) for _ in range(number)]

#dice(number=4, size=10) # WRONG: can not specify number as a keyword arg
#dice(4, 10) # WRONG: must specify size as a keyword
dice(4, size=10)

## *args and **kwargs

Now that you know what positional and keyword args are, we can talk about *args, and **kwargs in python. A common
convention in python code is to specify all positional arguments as `*args` and all keyword arguments as `**kwargs`.
While this is a common convention, the only place this should be used in modern python is when creating decorator 
functions (which will be a topic for a more advanced session).

- `*args` is actually a list of arguments that are passed to the function
- `**kwargs` is actually a dict, where the key is the name of the argument, and the value is whatever the value is.

The problem with `*args` and especially `**kwargs` is knowing what the meaning of the arguments are.  Sadly, a lot of
legacy code used these conventions and paid the price for it

> I used to work at Red Hat on the Openstack nova team (the compute side) and 9 years ago, it was 3 million lines of 
code.  We saw so many TypeError or KeyError bugs it wasn't funny.  This is because too many devs were lazy and just
created methods like `def foo(requird_arg1, *args, required_named_arg=10, **kwargs):`.  When you write code like this,
not only do you not know what the types are, you don't even know how many arguments are required, and what the key 
values (the names) in the `kwargs` dict are.  So, you are forced to look at the source code to figure out what is 
actually supposed to be passed in

In [ ]:
def bad_function(required_pos: int, *args, required_name: str, **kwargs):
    print(f"I am a required positional argument with value: {required_pos}")
    print(f"These are all the rest of the positional args: {args}")
    print(f"I am a required_named arg: {required_name}")
    named_args = [f"{k} = {v}" for k, v in kwargs.items()]
    print(f"and these are all the other named arguments: {named_args}")

# "hello" and "there" get stored in *args, and extra=10 and new_data=[1,2,3] get stored in kwargs
bad_function(12, "hello", "there", required_name="sean", extra=10, new_data=[1, 2, 3])

# example of ** unpacking with a dict
print("\nExample of ** unpacking")
obj = {"one": 1, "two": "TWO"}
bad_function(10, "hi", "sean", required_name="toner", **obj)

# example of * unpacking with a sequence and an empty kwargs
print("\nExample of * unpacking and an empty kwargs")
data = (1, 2, 3)
bad_function(4, *data, required_name="john")

print("\nWhat happens if you don't add the * to *data")
bad_function(4, data, required_name="john")

print("\nWhat happens if you add only a single * to obj")
bad_function(10, "hi", "sean", required_name="toner", *obj)

### When to use *args and **kwargs: Passing lists and dicts

Sometimes, the amount of arguments in a function becomes very large, and it is easier to pass in the arguments either
by a list (for positional args) or by a dict (for named arguments).  This is sometimes useful when you have a class
with many fields that need to be initialized as is often the case when using a `dataclass` type.

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Tuple


def lots_of_args(
    mapping: dict,
    answers: Tuple[int, int],
    cwd: str | None,
    completed: bool,
    start: datetime,
    end: datetime,
    timeout: timedelta
):
    print(mapping)
    print(answers)
    print(cwd)
    print(completed)
    print(start)
    print(end)
    print(timeout)
    return end + timeout

args = [
    {1: "foo"},
    (100, 200),
    None
]
kwargs = {
    "completed": True,
    "start": datetime.now(),
    "end": datetime.now(),
    "timeout": timedelta(hours=1)
}
lots_of_args(*args, **kwargs)


## Python @overload

A relatively new feature of python is the @overload decorator.  This is somewhat similar to how Java's or Mojo's 
function overloading works.  It's a little bit more painful, but does offer a useful feature.  It's most useful when
you have arguments that can be of multiple types and multiple return types, where the type of the return value is
dependent on the type of the argument(s).

### A brief note on Union types

Python's type system has the notion of a `Union` which means a type may either be one type or another.  Before 3.10,
you wrote Unions like this in 3.10+:

```python
def add_to_date(time: str | datetime, delta: timedelta) -> datetime:
    ...
```

Or this before 3.10

```python
from typing import Union

def add_to_date(time: Union[str, datetime], delta: timedelta) -> datetime:
    ...
```

The `time` argument can take _either_ a `str` or a `datetime`.

In [ ]:
import os
from pathlib import Path

# First, create the function that takes everything
def writer(content: str | bytes, output: str | Path | None = None):
    """Writes content to output path

    Parameters
    ----------
    content : str | bytes
        the data to write to file
    output : str | Path | None
        path of file to write to (if none, current directory)
    """
    # We defaulted to None, so we check for it
    if output is None:
        output = Path(os.getcwd()) / "testdata"
    
    # Example of pattern matching
    match content:
        case bytes() as _:    # if content is a bytes type
            mode = "wb"  # for binary add a 'b'
        case _:
            mode = "w+"
    with open(output, mode) as f: 
        f.write(content)

# Then, write an overloaded method for each possible overloaded argument


writer("just a test")  # No argument passed to `output` parameter
# Check your directory

test_path = Path("testdata2")
writer("another test", output=test_path)
bin_path = "binary_data"
writer(b'01234', bin_path)


In [ ]:
# delete data files
test_path.unlink()
Path("testdata").unlink()
Path("binary_data").unlink()

## Closures and Functions

Understanding scopes is necessary to make use of closures and nested functions.  We already showed several examples
of functions